## Deliverable 2. Create a Customer Travel Destinations Map.

In [4]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hasaki,JP,35.73,140.83,48.49,56,100,9.28,overcast clouds
1,1,Tsumeb,NaN,-19.23,17.72,74.75,25,35,11.41,scattered clouds
2,2,Kodiak,US,57.79,-152.41,20.71,93,90,3.44,overcast clouds
3,3,Mataura,NZ,-46.19,168.86,50.49,98,100,2.15,light rain
4,4,Hilo,US,19.73,-155.09,70.25,93,90,4.61,overcast clouds


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
7,7,Rikitea,PF,-23.12,-134.97,75.63,78,97,9.15,overcast clouds
13,13,Vaini,TO,-21.20,-175.20,78.96,83,40,8.05,scattered clouds
15,15,Annamalainagar,IN,11.40,79.73,84.70,69,19,7.34,few clouds
32,32,Busselton,AU,-33.65,115.33,84.09,40,72,21.09,broken clouds
34,34,Tambun,ID,-6.26,107.07,88.41,65,40,5.75,scattered clouds
35,35,Grand Gaube,MU,-20.01,57.66,85.37,59,2,10.00,clear sky
37,37,Butaritari,KI,3.07,172.79,82.98,80,100,17.22,light rain
38,38,Faanui,PF,-16.48,-151.75,81.41,62,69,11.86,broken clouds
41,41,Carnarvon,AU,-24.87,113.63,82.47,65,2,12.66,clear sky
45,45,Avarua,CK,-21.21,-159.78,78.85,89,100,4.61,overcast clouds


In [8]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()


City_ID                174
City                   174
Country                174
Lat                    174
Lng                    174
Max Temp               174
Humidity               174
Cloudiness             174
Wind Speed             174
Current Description    174
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()


City_ID                174
City                   174
Country                174
Lat                    174
Lng                    174
Max Temp               174
Humidity               174
Cloudiness             174
Wind Speed             174
Current Description    174
dtype: int64

In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,Rikitea,PF,75.63,overcast clouds,-23.12,-134.97,
13,Vaini,TO,78.96,scattered clouds,-21.20,-175.20,
15,Annamalainagar,IN,84.70,few clouds,11.40,79.73,
32,Busselton,AU,84.09,broken clouds,-33.65,115.33,
34,Tambun,ID,88.41,scattered clouds,-6.26,107.07,
35,Grand Gaube,MU,85.37,clear sky,-20.01,57.66,
37,Butaritari,KI,82.98,light rain,3.07,172.79,
38,Faanui,PF,81.41,broken clouds,-16.48,-151.75,
41,Carnarvon,AU,82.47,clear sky,-24.87,113.63,
45,Avarua,CK,78.85,overcast clouds,-21.21,-159.78,


In [11]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        print(hotels["results"][0]["name"])
    except (IndexError):
        print("Hotel not found... skipping.")    
        

People ThankYou
Keleti Beach Resort
Hotel Akshaya
Observatory Guest House
Metland Hotel Bekasi
Veranda Paul et Virginie Hotel & Spa
Isles Sunset Lodge
Bora-Bora Pearl Beach Resort & Spa
Hospitality Carnarvon
Paradise Inn
VS Villa
Pousada Sitio Do Carmo
Villa Enata
Le Refuge
Casa Tia Villa
Aldea del Bazar Hotel and Spa
Lidera
ALETHEIA Appartement Sinnamary
Melrose
Shelly Bay Resort
Scoop Guest House
Pausada Morabedja
Hotel not found... skipping.
Student Campus 2
Axim Beach Resort And Spa
"Plantation Bed and Breakfast
Hotel Mpampa
Eng Hiirey House
Bils' Villa
Maringe Lagoon Lodge
Khách sạn Sài Gòn Bạc Liêu
Sea Lounge! Bar, Club & Restaurant
Atlantis Hotel
Grage Hotel Bengkulu
Daru Lodge
Hospedaje la Amistad
Al Mumtaz Transport Jiwani
Hotel not found... skipping.
Casa Dora, Bubaque
HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel
Mystic Sands Beachfront Accommodation
Hotel Fanilo
Taloa Heights
Hotel Olako
Labuhan
Sulthan Hotel International
Bustani B&B
Butterfly Beach Hotel
Osprey Beach Hotel
SEG

In [12]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df
clean_hotel_df.drop(clean_hotel_df[clean_hotel_df['Hotel Name'] == ''].index, inplace = True)
#clean_hotel_df.count()

clean_hotel_df.head(50)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,Rikitea,PF,75.63,overcast clouds,-23.12,-134.97,People ThankYou
13,Vaini,TO,78.96,scattered clouds,-21.20,-175.20,Keleti Beach Resort
15,Annamalainagar,IN,84.70,few clouds,11.40,79.73,Hotel Akshaya
32,Busselton,AU,84.09,broken clouds,-33.65,115.33,Observatory Guest House
34,Tambun,ID,88.41,scattered clouds,-6.26,107.07,Metland Hotel Bekasi
35,Grand Gaube,MU,85.37,clear sky,-20.01,57.66,Veranda Paul et Virginie Hotel & Spa
37,Butaritari,KI,82.98,light rain,3.07,172.79,Isles Sunset Lodge
38,Faanui,PF,81.41,broken clouds,-16.48,-151.75,Bora-Bora Pearl Beach Resort & Spa
41,Carnarvon,AU,82.47,clear sky,-24.87,113.63,Hospitality Carnarvon
45,Avarua,CK,78.85,overcast clouds,-21.21,-159.78,Paradise Inn


In [13]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")


In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [17]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.4)
#heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
#            max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
#fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))